In [1]:
# create_table.ipynb

import excel_like_table as elt
from datetime import datetime

# Sample input
input_data = [
    {"name": "Alice", "age": 30, "birthday": datetime(1993, 5, 17, 14, 30), "salary": 60000.00, "city": ["New York", "Los Angeles"], "meetings": [datetime(2023, 5, 28, 10, 0), datetime(2023, 6, 15, 14, 0)]},
    {"name": "Bob", "age": 25, "birthday": datetime(1998, 8, 24, 9, 15), "salary": 50000.50, "city": ["San Francisco", "Seattle"], "meetings": [datetime(2023, 5, 30, 16, 0), datetime(2023, 6, 20, 11, 0)]}
]

# Create the table
elt.create_table(input_data)

In [2]:
# Load the saved table data
saved_data = elt.load_saved_table()
print(saved_data)

[{'name': 'Alice', 'age': 30, 'birthday': '1993-05-17T14:30:00', 'salary': 60000.0, 'city': 'New York,Los Angeles', 'meetings': '2023-05-28T10:00:00,2023-06-15T14:00:00'}, {'name': 'Bob', 'age': 25, 'birthday': '1998-08-24T09:15:00', 'salary': 50000.5, 'city': 'San Francisco,Seattle', 'meetings': '2023-05-30T16:00:00,2023-06-20T11:00:00'}, {'name': 'Bob', 'age': 25, 'birthday': '1998-08-24T09:15:00', 'salary': 50000.5, 'city': 'San Francisco,Seattle', 'meetings': '2023-05-30T16:00:00,2023-06-20T11:00:00'}]
